In [ ]:
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 25.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('merged_dataset_new.csv')

In [ ]:
df['Label_bit'] = df.apply(lambda row: 1 if row['Label'] == 'Entailment' else 0, axis=1)

In [ ]:
df.value_counts('Label_bit')

Label_bit
0    850
1    850
Name: count, dtype: int64

In [ ]:
df.head(10)

,test_ID,Type,Section_id,Primary_id,Secondary_id,Statement,Label,Primary_evidence_index,Secondary_evidence_index,Reference_data,Label_bit
0,5bc844fc-e852-4270-bfaf-36ea9eface3d,Comparison,Intervention,NCT01928186,NCT00684983,All the primary trial participants do not rece...,Contradiction,INTERVENTION 1: Diagnostic (FLT PET) Pati...,INTERVENTION 1: Arm A Patients receive or...,INTERVENTION 1: Diagnostic (FLT PET) Pati...,0
1,86b7cb3d-6186-4a04-9aa6-b174ab764eed,Single,Eligibility,NCT00662129,NaN,"Patients with Platelet count over 100,000/mm¬¨...",Contradiction,"PATIENT CHARACTERISTICS: ANC 1,500/mm³ ...",NaN,DISEASE CHARACTERISTICS: Histologically or c...,0
2,dbed5471-c2fc-45b5-b26f-430c9fa37a37,Comparison,Adverse Events,NCT00093145,NCT00703326,Heart-related adverse events were recorded in ...,Entailment,Adverse Events 1: Supraventricular tachycard...,Adverse Events 1: Atrial fibrillation 1/752 ...,Adverse Events 1: Total: 5/32 (15.63%) Feb...,1
3,20c35c89-8d23-4be3-b603-ac0ee0f3b4de,Single,Eligibility,NCT01097642,NaN,Adult Patients with histologic confirmation of...,Contradiction,Inclusion Criteria: Patients with histologic...,NaN,Inclusion Criteria: Patients with histologic...,0
4,f17cb242-419d-4f5d-bfa4-41494ed5ac0e,Comparison,Intervention,NCT00852930,NCT02308020,Laser Therapy is in each cohort of the primary...,Contradiction,INTERVENTION 1: Laser Therapy Alone thera...,"INTERVENTION 1: Part A Abemaciclib: HR+, HE...",INTERVENTION 1: Laser Therapy Alone thera...,0
5,fc5c4554-7ce9-4c16-b374-a3cd9d15b021,Comparison,Eligibility,NCT00971945,NCT01027416,Patients must have already participated in a s...,Contradiction,Inclusion Criteria: Subjects who were confir...,Inclusion Criteria: The patient must consent...,Inclusion Criteria: Subjects who were confir...,0
6,96b77cdd-aa9f-4770-8447-8a04d9ca5da7,Single,Eligibility,NCT00633750,NaN,Patients with Clinical stage II (T2 N1) invasi...,Contradiction,Inclusion Criteria: Clinical stage I or II (...,NaN,Inclusion Criteria: Clinical stage I or II (...,0
7,c73faed2-371b-4238-bf7d-293fae380203,Comparison,Intervention,NCT00003404,NCT00711529,the primary trial and the secondary trial have...,Contradiction,INTERVENTION 1: Adjuvant Radiotherapy Adj...,INTERVENTION 1: Hypnotherapy Patients ran...,INTERVENTION 1: Adjuvant Radiotherapy Adj...,0
8,8765009d-ffc4-4395-ab7a-11ecdfd43a40,Single,Eligibility,NCT00201773,NaN,Adele is an 85 year old woman with Stage II hi...,Entailment,Inclusion Criteria: Must be female with hist...,NaN,Inclusion Criteria: Must be female with hist...,1
9,0ad7293d-df35-42e8-881d-f2afc3f7d3fd,Comparison,Intervention,NCT02606708,NCT02504424,Only patients in the primary trial receive 40....,Contradiction,INTERVENTION 1: Accelerated Intensity Modul...,INTERVENTION 1: AeroForm Tissue Expander ...,INTERVENTION 1: Accelerated Intensity Modul...,0


In [ ]:
data = df[['Type', 'Statement', 'Reference_data', 'Label_bit']]

In [ ]:
data.head()

,Type,Statement,Reference_data,Label_bit
0,Comparison,All the primary trial participants do not rece...,INTERVENTION 1: Diagnostic (FLT PET) Pati...,0
1,Single,"Patients with Platelet count over 100,000/mm¬¨...",DISEASE CHARACTERISTICS: Histologically or c...,0
2,Comparison,Heart-related adverse events were recorded in ...,Adverse Events 1: Total: 5/32 (15.63%) Feb...,1
3,Single,Adult Patients with histologic confirmation of...,Inclusion Criteria: Patients with histologic...,0
4,Comparison,Laser Therapy is in each cohort of the primary...,INTERVENTION 1: Laser Therapy Alone thera...,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Type            1700 non-null   object
 1   Statement       1700 non-null   object
 2   Reference_data  1700 non-null   object
 3   Label_bit       1700 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 53.2+ KB


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')
model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.1', num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
model = model.to('cuda')

In [ ]:
sample_data = ["I am eating", "I am playing"]
tokenizer(sample_data, padding=True, truncation=True, max_length = 512)

{'input_ids': [[101, 178, 1821, 5497, 102], [101, 178, 1821, 1773, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [ ]:
data['combined'] = "Type: " + data['Type'] + " Statement: " + data['Statement'] + " Reference_data: " + data['Reference_data']
X = list(data['combined'])
y = list(data['Label_bit'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length = 512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length = 512)

<ipython-input-10-7125cf05275b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['combined'] = "Type: " + data['Type'] + " Statement: " + data['Statement'] + " Reference_data: " + data['Reference_data']


In [ ]:
X_train[0]

'Type: Comparison Statement: There were 0 patients with Dysphagia or Fever in the primary trial and the secondary trial cohorts. Reference_data: Adverse Events 1:   Total: 3/5 (60.00%)   Febrile neutropenia 3/5 (60.00%)   Atrial fibrillation 0/5 (0.00%)   Myocardial Infarction 0/5 (0.00%)   Blurred Vision 0/5 (0.00%)   Dysphagia 0/5 (0.00%)   Fever 0/5 (0.00%)   General disorders and administration site conditions - Other 0/5 (0.00%)   Localized edema 0/5 (0.00%)   Non-cardiac chest pain 0/5 (0.00%)   Pain 0/5 (0.00%)   Sepsis 0/5 (0.00%) Adverse Events 1:   Total: 17/110 (15.45%)   Hemoglobin  [1]1/110 (0.91%)   Esophagitis 1/110 (0.91%)   Dysphagia 1/110 (0.91%)   Nausea/Vomiting 1/110 (0.91%)   Nausea 1/110 (0.91%)   Fever 1/110 (0.91%)   Febrile Neutropenia 11/110 (10.00%)   Infection - Other  [2]1/110 (0.91%)   Infection - Pneumonia 1/110 (0.91%)   Dyspnea 2/110 (1.82%)   Hypoxia 1/110 (0.91%)'

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
len(X_train), len(X_val)

(1275, 425)

In [ ]:

class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels = None):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    if self.labels:
      item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings['input_ids'])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
train_dataset[5]

{'input_ids': tensor([  101,  2076,   131,  7577,  4195,   131,  1884, 13252,  1204,   122,
          1104,  1103,  2425,  3443,  1105,  1884, 13252,  1204,   122,  1104,
          1103,  3718,  3443,  1138,  1103,  1269,  1703,  1295,  1104, 16798,
          1958,   119,  3835,   168,  2233,   131, 16798,  1958,   122,   131,
          1703,   131,  1405,   120,  3882,   113,  1743,   119,  4859,   110,
           114,  1126,  5024,  8191,   115,   122,   120,  3882,   113,   122,
           119,  5556,   110,   114,   175, 15581, 13217,  1162, 24928,  3818,
         12736, 23179,  1572,   120,  3882,   113,   128,   119,  5691,   110,
           114, 24928,  3818, 12736, 23179,  1743,   120,  3882,   113,  1405,
           119,  3383,   110,   114, 17688,  4290, 13306,  1626,   120,  3882,
           113,   122,   119,  5556,   110,   114, 26979,  1158,  1695,   120,
          3882,   113,   126,   119,  5581,   110,   114,  4267,  1813,  1197,
         10061,  1161,  1626,   120,  3

In [ ]:
def compute_metrics(p):
  print(type(p))
  pred, labels = p
  pred = np.argmax(pred, axis=1)
  accuracy = accuracy_score(y_true=labels, y_pred=pred)
  recall = recall_score(y_true=labels, y_pred=pred)
  precision = precision_score(y_true=labels, y_pred=pred)
  f1 = f1_score(y_true=labels, y_pred=pred)
  return {"accuracy": accuracy, "recall": recall, "precision": precision, "f1": f1}

In [ ]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
!pip install accelerate==0.21.0
import accelerate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install --upgrade transformers accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


In [ ]:

from transformers import Trainer, TrainingArguments
import torch

args = TrainingArguments(
    output_dir='output',
    num_train_epochs=25,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,No log,0.693938,0.498824,0.772947,0.490798,0.600375
2,No log,0.693347,0.508235,0.299517,0.492063,0.372372
3,No log,0.692382,0.517647,0.449275,0.505435,0.475703
4,No log,0.695256,0.512941,0.000000,0.000000,0.000000
5,No log,0.695187,0.487059,1.000000,0.487059,0.655063
6,No log,0.704717,0.487059,1.000000,0.487059,0.655063
7,0.703700,0.698703,0.487059,1.000000,0.487059,0.655063
8,0.703700,0.694761,0.512941,0.000000,0.000000,0.000000
9,0.703700,0.695291,0.498824,0.048309,0.384615,0.085837
10,0.703700,0.696338,0.477647,0.932367,0.481297,0.634868


<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>


TrainOutput(global_step=2000, training_loss=0.5802243041992188, metrics={'train_runtime': 751.3806, 'train_samples_per_second': 42.422, 'train_steps_per_second': 2.662, 'total_flos': 8386664889600000.0, 'train_loss': 0.5802243041992188, 'epoch': 25.0})

In [ ]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.692381739616394,
 'eval_accuracy': 0.5176470588235295,
 'eval_recall': 0.4492753623188406,
 'eval_precision': 0.5054347826086957,
 'eval_f1': 0.4757033248081841,
 'eval_runtime': 2.757,
 'eval_samples_per_second': 154.154,
 'eval_steps_per_second': 9.793,
 'epoch': 25.0}

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
df_predict = pd.read_csv('Gold_test_dataset.csv')

In [ ]:
df_predict.head()

,test_ID,Type,Section_id,Primary_id,Secondary_id,Statement,Label,Primary_evidence_index,Secondary_evidence_index,Reference_data
0,9f978634-637c-472f-a588-6f4bb2fb121f,Comparison,Eligibility,NCT01129622,NCT01156987,Women suffering from both claustrophobia and I...,Contradiction,Exclusion Criteria: History of bilateral mas...,Exclusion Criteria: Severe claustrophobia,Inclusion Criteria: Women are eligible to pa...
1,20b34e62-97c2-4ca0-bb1d-7824dab0b8bb,Single,Eligibility,NCT00322374,NaN,There are no conditions on mental mental healt...,Entailment,Inclusion Criteria: Women 18 years Histolo...,NaN,Inclusion Criteria: Women 18 years Histolo...
2,893a5337-2aa9-4a87-a020-4c2f03cd4aea,Comparison,Eligibility,NCT00122369,NCT03069313,the secondary trial requires patients to be ov...,Entailment,Inclusion Criteria: Patients presenting for ...,Inclusion Criteria: Sign informed consent an...,Inclusion Criteria: Patients presenting for ...
3,d401affc-f081-4eee-bd61-d109cc88f6de,Comparison,Eligibility,NCT04080297,NCT00929240,Patients currently taking part in the secondar...,Entailment,Participation in another clinical trial with...,"Inclusion Criteria: adult patients, >=18 yea...",Inclusion Criteria: Be a female of any race ...
4,791790a6-187b-4e4b-be5f-9e5304e9ec2c,Comparison,Intervention,NCT01301729,NCT00021255,HER2 + Patients in the primary trial receive t...,Contradiction,INTERVENTION 1: Trastuzumab Participants ...,INTERVENTION 2: AC Followed by Docetaxel + ...,INTERVENTION 1: Trastuzumab Participants ...


In [ ]:
df_predict['combined'] = "Type: " + df_predict['Type'] + " Statement: " + df_predict['Statement'] + " Reference_data: " + df_predict['Reference_data']

In [ ]:
test_text = df_predict['combined'][1]

In [ ]:
test_text

'Type: Single Statement: There are no conditions on mental mental health, bodyweight, size of tumours, Karnofsky/ECOG score or Estimated life expectancy that need to be met in order to be eligible for the primary trial  Reference_data: Inclusion Criteria:   Women 18 years   Histologically or cytologically confirmed diagnosis of metastatic breast cancer   Measurable or nonmeasurable disease defined by Response Evaluation Criteria In Solid Tumors (RECIST) Exclusion Criteria:   Number of prior chemotherapy lines of treatment in the metastatic setting 2'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')
model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.1', num_labels=2)
inputs = tokenizer(test_text, padding=True, truncation=True, max_length=512, return_tensors='pt')

outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(outputs)
print(predictions)

predictions = predictions.cpu().detach().numpy()
print(predictions)
if predictions[0][0]>predictions[0][1]:
  print('Contradiction')
else:
  print('Entailment')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SequenceClassifierOutput(loss=None, logits=tensor([[-0.4600, -0.1008]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.4111, 0.5889]], grad_fn=<SoftmaxBackward0>)
[[0.41114265 0.5888573 ]]
Entailment


In [ ]:
df_predict['Label'][1]

'Entailment'

In [ ]:
trainer.save_model('BioBERT_Label_Prediction')

In [ ]:
import shutil
shutil.make_archive('BioBERT_Label_Prediction', 'zip', 'BioBERT_Label_Prediction')


'/content/BioBERT_Label_Prediction.zip'

In [ ]:
# Generating the prediction labels

In [ ]:
!pip install transformers accelerate torch scikit-learn datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d322e0ab3b08582809639b35b386b5ab92ccb615daccfcc5cf234334dde9187f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_metric
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

rouge = load_metric('rouge')

In [ ]:

data = pd.DataFrame({
    'Type': ['Type1', 'Type2'],
    'Statement': ['Statement1', 'Statement2'],
    'Reference_data': ['Reference1', 'Reference2'],
    'Primary_evidence_index': ['Primary1', 'Primary2'],
    'Secondary_evidence_index': ['Secondary1', 'Secondary2']
})


data['combined'] = "Type: " + data['Type'] + " Statement: " + data['Statement'] + " Reference_data: " + data['Reference_data']

data['target'] = data['Primary_evidence_index'] + " " + data['Secondary_evidence_index']


X_train, X_val, y_train, y_val = train_test_split(data['combined'], data['target'], test_size=0.25, random_state=42)


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, targets, tokenizer, max_length=512):
        self.encodings = encodings
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, idx):
        input_encoding = self.tokenizer(self.encodings[idx], max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        target_encoding = self.tokenizer(self.targets[idx], max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")

        input_encoding = {key: val.squeeze(0) for key, val in input_encoding.items()}
        target_encoding = {key: val.squeeze(0) for key, val in target_encoding.items()}

        input_encoding['labels'] = target_encoding['input_ids']

        return input_encoding

    def __len__(self):
        return len(self.encodings)


tokenizer = T5Tokenizer.from_pretrained('t5-small')

train_dataset = CustomDataset(X_train.tolist(), y_train.tolist(), tokenizer)
val_dataset = CustomDataset(X_val.tolist(), y_val.tolist(), tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple):
        logits = logits[0]

    predictions = np.argmax(logits, axis=-1)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(pred.strip() for pred in preds.split()) for preds in decoded_preds]
    decoded_labels = ["\n".join(label.strip() for label in labels.split()) for labels in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return result


In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
model = model.to('cuda')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="rouge1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,25.186588,0.000000,0.000000,0.000000,0.000000,512.000000
2,No log,24.540564,0.000000,0.000000,0.000000,0.000000,512.000000
3,No log,23.972006,0.000000,0.000000,0.000000,0.000000,512.000000
4,No log,23.375469,0.000000,0.000000,0.000000,0.000000,512.000000
5,No log,22.632580,0.000000,0.000000,0.000000,0.000000,512.000000
6,No log,21.887041,0.000000,0.000000,0.000000,0.000000,512.000000
7,No log,21.191093,0.000000,0.000000,0.000000,0.000000,512.000000
8,No log,20.543631,0.000000,0.000000,0.000000,0.000000,512.000000
9,No log,19.938929,0.000000,0.000000,0.000000,0.000000,512.000000
10,No log,19.431343,0.000000,0.000000,0.000000,0.000000,512.000000


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'eval_loss': 25.186588287353516,
 'eval_rouge1': 0.0,
 'eval_rouge2': 0.0,
 'eval_rougeL': 0.0,
 'eval_rougeLsum': 0.0,
 'eval_gen_len': 512.0,
 'eval_runtime': 0.3423,
 'eval_samples_per_second': 2.922,
 'eval_steps_per_second': 2.922,
 'epoch': 20.0}

In [ ]:
def generate_sentence(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to('cuda') for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

example_input = df_predict['combined'][1]
generated_sentence = generate_sentence(example_input)
print(generated_sentence)


: Single Statement: There are no conditions on mental mental health, bodyweight, size of tumours, Karnofsky/ECOG score or Estimated life expectancy that need to be met. Referenz_data: Inclusion Criteria: Women 18 years Histologically or cytologically confirmed diagnosis of metastatic breast cancer Measurable or nonmeasurable disease defined by Response Evaluation Criteria In Solid Tumors (RECIST) Exclusion Criteria: Number of prior chemotherapy lines of treatment in the metastatic


In [ ]:
df_predict['Primary_evidence_index'][1]

'Inclusion Criteria:   Women 18 years   Histologically or cytologically confirmed diagnosis of metastatic breast cancer   Measurable or nonmeasurable disease defined by Response Evaluation Criteria In Solid Tumors (RECIST) Exclusion Criteria:   Number of prior chemotherapy lines of treatment in the metastatic setting 2'

In [ ]:
df_predict['Secondary_evidence_index'][1]

nan

In [ ]:
trainer.save_model('Reference_Generator')

In [ ]:
import shutil
shutil.make_archive('Reference_Generator', 'zip', 'Reference_Generator')


'/content/Reference_Generator.zip'

In [ ]:
from google.colab import files
files.download('Reference_Generator.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>